<a href="https://colab.research.google.com/github/bashar-z/Youtube-Clipper-Tool/blob/main/Clipper_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ====== YouTube Clip Tool (Colab version with time pickers) ======
!pip install -q yt-dlp ipywidgets
!apt-get -y install -q ffmpeg

import subprocess, glob
from yt_dlp import YoutubeDL
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Widgets ---
url_box = widgets.Text(
    description='🎥 YouTube URL:',
    placeholder='Paste YouTube link here...',
    layout=widgets.Layout(width='90%')
)

# Time pickers for start
start_h = widgets.BoundedIntText(value=0, min=0, max=23, description='Start H:')
start_m = widgets.BoundedIntText(value=0, min=0, max=59, description='M:')
start_s = widgets.BoundedIntText(value=0, min=0, max=59, description='S:')

# Time pickers for end
end_h = widgets.BoundedIntText(value=0, min=0, max=23, description='End H:')
end_m = widgets.BoundedIntText(value=0, min=0, max=59, description='M:')
end_s = widgets.BoundedIntText(value=0, min=0, max=59, description='S:')

clip_button = widgets.Button(description='🎬 Clip & Download', button_style='success')
output_box = widgets.Output()

time_grid = widgets.HBox([
    widgets.VBox([start_h, start_m, start_s]),
    widgets.VBox([end_h, end_m, end_s])
])

def on_clip_button_click(b):
    clear_output(wait=True)
    display(url_box, time_grid, clip_button, output_box)
    with output_box:
        yt_url = url_box.value.strip()
        start = f"{start_h.value:02d}:{start_m.value:02d}:{start_s.value:02d}"
        end = f"{end_h.value:02d}:{end_m.value:02d}:{end_s.value:02d}"

        if not yt_url:
            print("⚠️ Please enter a YouTube link.")
            return

        print(f"⬇️ Downloading video…\n")
        ydl_opts = {
            "format": 'bv*[ext=mp4]+ba[ext=m4a]/b[ext=mp4]/bv*+ba/b',
            "outtmpl": "full_video.%(ext)s",
            "merge_output_format": "mp4",
            "quiet": True,
        }

        try:
            with YoutubeDL(ydl_opts) as ydl:
                ydl.extract_info(yt_url, download=True)
        except Exception as e:
            print("❌ Download error:", e)
            return

        files_found = sorted(glob.glob("full_video.*"), key=len)
        if not files_found:
            print("❌ Could not find downloaded file.")
            return

        src = files_found[0]
        if not src.endswith(".mp4"):
            subprocess.run(["ffmpeg","-y","-i",src,"-c","copy","full_video.mp4"], check=True)
            src = "full_video.mp4"

        clip_name = f"clip_{start.replace(':','-')}_{end.replace(':','-')}.mp4"
        print(f"✂️  Clipping from {start} to {end} …\n")

        try:
            subprocess.run([
                "ffmpeg","-y","-ss",start,"-to",end,"-i",src,
                "-c","copy",clip_name
            ], check=True)
        except subprocess.CalledProcessError:
            print("❌ FFmpeg failed while clipping.")
            return

        print(f"✅ Done! Saved as: {clip_name}")
        files.download(clip_name)

display(url_box, time_grid, clip_button, output_box)
clip_button.on_click(on_clip_button_click)


Text(value='https://www.youtube.com/watch?v=TvfgJAdehPg', description='🎥 YouTube URL:', layout=Layout(width='9…

Button(button_style='success', description='🎬 Clip & Download', style=ButtonStyle())

Output()